In [11]:
import tensorflow as tf 
import numpy as np 
import math
#import tensorflow.contrib.skflow as skflow
from tensorflow.contrib.learn.python import learn as skflow

In [12]:
import sklearn
from sklearn import metrics 

In [13]:
try:
    from tqdm import tqdm
except ImportError:
    def tqdm(x,*args,**kwargs ):
        return x

In [14]:
np.random.seed(0)

In [34]:
data = np.load('data_with_labels.npz')
train  = data['arr_0']/255.
labels = data['arr_1']
print(train[0])
print(labels[0])

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
0


In [35]:
import matplotlib.pyplot as plt
plt.ion()

In [37]:
indices = np.random.permutation(train.shape[0])
valid_cnt = int(train.shape[0]*0.1)
test_idx,training_idx = indices[:valid_cnt], indices[valid_cnt:]
test,train = train[test_idx,:],train[training_idx,:]
test_labels,train_labels = labels[test_idx], labels[training_idx]

In [38]:
sess = tf.InteractiveSession()

In [52]:
feature_columns = skflow.infer_real_valued_columns_from_input(train)
classifier = skflow.LinearClassifier(n_classes = 5,optimizer='Adam',feature_columns=feature_columns,enable_centered_bias=True)

In [53]:
classifier.fit(train.reshape([-1,36*36]),labels.astype(np.int32),steps=1000)

Estimator(params={'joint_weights': False, 'gradient_clip_norm': None, 'num_ps_replicas': 0, 'optimizer': <tensorflow.python.training.adam.AdamOptimizer object at 0x11b2b60b8>, 'n_classes': 5, 'weight_column_name': None, 'enable_centered_bias': True, 'feature_columns': [_RealValuedColumn(column_name='', dimension=1296, default_value=None, dtype=tf.float64, normalizer=None)]})

In [54]:
 sklearn.metrics.accuracy_score(test_labels,classifier.predict(test.reshape([-1,36*36])))

Instructions for updating:
The default behavior of predict() is changing. The default value for
as_iterable will change to True, and then the flag will be removed
altogether. The behavior of this flag is described below.


0.2078853046594982

In [59]:
classifier2 = skflow.DNNClassifier(hidden_units=[10,5],n_classes=5,optimizer='Adam',feature_columns=feature_columns,enable_centered_bias=True)
classifier2.fit(train.reshape([-1,36*36]),labels.astype(np.int32),steps=1000)

Estimator(params={'activation_fn': <function relu at 0x1119f5730>, 'gradient_clip_norm': None, 'hidden_units': [10, 5], 'num_ps_replicas': 0, 'optimizer': 'Adam', 'n_classes': 5, 'weight_column_name': None, 'enable_centered_bias': True, 'feature_columns': [_RealValuedColumn(column_name='', dimension=1296, default_value=None, dtype=tf.float64, normalizer=None)], 'dropout': None})

In [60]:
sklearn.metrics.accuracy_score(test_labels,classifier2.predict(test.reshape([-1,36*36])))

Instructions for updating:
The default behavior of predict() is changing. The default value for
as_iterable will change to True, and then the flag will be removed
altogether. The behavior of this flag is described below.
Instructions for updating:
The default behavior of predict() is changing. The default value for
as_iterable will change to True, and then the flag will be removed
altogether. The behavior of this flag is described below.


0.2078853046594982

In [62]:
conf = metrics.confusion_matrix(train_labels,classifier2.predict(train.reshape([-1,36*36])))
print(conf)

Instructions for updating:
The default behavior of predict() is changing. The default value for
as_iterable will change to True, and then the flag will be removed
altogether. The behavior of this flag is described below.
Instructions for updating:
The default behavior of predict() is changing. The default value for
as_iterable will change to True, and then the flag will be removed
altogether. The behavior of this flag is described below.


[[  3   0   0 499   0]
 [  0   0   0 492   0]
 [  0   0   0 507   0]
 [  0   0   0 501   0]
 [  0   0   0 509   0]]


In [63]:
def conv_skflow(X, y):
    X = tf.reshape(X, [-1, 36, 36, 1])
    # conv layer will compute 4 kernels for each 5x5 patch
    with tf.variable_scope('conv_layer'):
        # 5x5 convolution, pad with zeros on edges
        h1 = skflow.ops.conv2d(X, n_filters=4,
                filter_shape=[5, 5], 
                bias=True, activation=tf.nn.relu)
        # 2x2 Max pooling, no padding on edges
        p1 = tf.nn.max_pool(h1, ksize=[1, 2, 2, 1],
                strides=[1, 2, 2, 1], padding='VALID')

    # Need to flatten conv output for use in dense layer
    p1_size = np.product(
              [s.value for s in p1.get_shape()[1:]])
    p1f = tf.reshape(p1, [-1, p1_size ])

    # densely connected layer with 32 neurons and dropout
    h_fc1 = skflow.ops.dnn(p1f,
            [32],
            activation=tf.nn.relu,
            dropout=0.5)
    return skflow.models.logistic_regression(h_fc1, y)

In [64]:
classifier3 = skflow.TensorFlowEstimator(model_fn=conv_skflow,n_classes=5,optimizer='Adam')

In [65]:
classifier3.fit(train,train_labels)

AttributeError: module 'tensorflow.contrib.learn.python.learn.ops' has no attribute 'conv2d'